In [2]:
#import networkx as nx
import csv
import osmnx as ox
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib;
import pandas as pd 
import utm
import matplotlib.pyplot as plt 
from shapely.geometry import LineString, mapping
import geopandas as gpd
from ipyleaflet import *
from ipyleaflet import Map, FullScreenControl, Polyline
from datetime import datetime, timedelta, timezone
import subprocess
import numpy as np
ox.config(use_cache=True, log_console=True)
%matplotlib inline
from shapely.geometry import Point
import pyproj, math

# Read ground truth data
with open("../dataset/gps_solution_TST/groundTruth_TST.csv", 'r') as f:
    reader = csv.reader(f, delimiter=",",quoting=csv.QUOTE_NONNUMERIC);
    data_gt_raw = list(reader);
    
# Save ground truth using UTM conversion
with open("./results/gt_utm.tum", 'w') as f:
    dt_gps = datetime(1980, 1, 6).replace(tzinfo=timezone.utc)
    leap_seconds = 37 - 19 # 19 - 1980, 37 - 2021
    for item in data_gt_raw:
        # [0]-GPS week, [1]-GPS seconds, [2]-latitude, [3]-longirude, [4]-altitude
        timestamp = datetime.timestamp(dt_gps + timedelta(seconds = item[0]*3600*24*7+item[1] - leap_seconds))
        utm_res = utm.from_latlon(item[2], item[3]) # return easting, northing, zone
        f.write("{} {} {} {} {} {} {} {}\n".format(timestamp, utm_res[0], utm_res[1], item[4], 0, 0, 0, 1))

# Save ground truth using ECEF conversion
# EPSG:4978-ECEF, EPSG:4326-Lat,Lon , EPSG:32650-UTM 50N (Hong Kong)           
with open("./results/gt_ecef.tum", 'w') as f:
    data_gt_lat = [item[2] for item in data_gt_raw]
    data_gt_lon = [item[3] for item in data_gt_raw]
    data_gt_alt = [item[4] for item in data_gt_raw]
    data_gt_gpd = gpd.points_from_xy(data_gt_lon, data_gt_lat, z=data_gt_alt, crs="EPSG:4326")
    data_gt_gpd_conv = data_gt_gpd.to_crs(crs="EPSG:4978")   
    dt_gps = datetime(1980, 1, 6).replace(tzinfo=timezone.utc)
    leap_seconds = 37 - 19 # 19 - 1980, 37 - 2021
    for i, item in enumerate(data_gt_raw):
        # [0]-GPS week, [1]-GPS seconds, [2]-latitude, [3]-longirude, [4]-altitude
        timestamp = datetime.timestamp(dt_gps + timedelta(seconds = item[0]*3600*24*7+item[1] - leap_seconds))
        f.write("{} {} {} {} {} {} {} {}\n".format(timestamp, data_gt_gpd_conv[i].x, data_gt_gpd_conv[i].y, data_gt_gpd_conv[i].z, 0, 0, 0, 1))

# Read GPS estimated pose
with open("../dataset/gps_solution_TST/results.pos", 'r') as f:
    reader = csv.reader(filter(lambda row: row[0]!='%', f), delimiter=",",quoting=csv.QUOTE_NONNUMERIC)
    data_gps_raw = list(reader)

# Convert GPS estimated pose and save to file
with open("./results/gps_ecef.tum", 'w') as f:
    data_gps_lat = [item[2] for item in data_gps_raw]
    data_gps_lon = [item[3] for item in data_gps_raw]
    data_gps_alt = [item[4] for item in data_gps_raw]
    data_gps_gpd = gpd.points_from_xy(data_gps_lon, data_gps_lat, z=data_gps_alt, crs="EPSG:4326")
    data_gps_gpd_conv = data_gps_gpd.to_crs(crs="EPSG:4978")  
    dt_gps = datetime(1980, 1, 6).replace(tzinfo=timezone.utc)
    leap_seconds = 37 - 19 # 19 - 1980, 37 - 2021
    for i, item in enumerate(data_gps_raw):
        # [0]-GPS week, [1]-GPS seconds, [2]-latitude, [3]-longirude, [4]-altitude
        timestamp = datetime.timestamp(dt_gps + timedelta(seconds = item[0]*3600*24*7+item[1] - leap_seconds))
        f.write("{} {} {} {} {} {} {} {}\n".format(timestamp, data_gps_gpd_conv[i].x, data_gps_gpd_conv[i].y, data_gps_gpd_conv[i].z, 0, 0, 0, 1))

# Call evo_ape evaluation
f = open("./results/evo_ape.txt", "w")
subprocess.call("rm ape.pdf", shell=True, cwd="./results")
subprocess.call("evo_ape tum gt_ecef.tum gps_ecef.tum -va --save_plot ape.pdf", stdout=f, shell=True, cwd="./results")
f.close()

# Add map visualization
# Ground truth
gt_line = Polyline(
    locations=[[x,y] for week,sec,x,y,z in data_gt_raw],
    color="black",
    fill=False,
    opacity = 0.8
)
gps_line= Polyline(
    locations=[[x,y] for x,y in zip(data_gps_lat,data_gps_lon)],
    color="green",
    fill=False,
    opacity=0.8,
)
m = Map(center=[data_gt_lat[0], data_gt_lon[0]], basemap=basemaps.OpenStreetMap.Mapnik, zoom=15)
m.add_control(FullScreenControl())
m.add_layer(gt_line)
m.add_layer(gps_line)
m

Map(center=[22.330486, 114.17954], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…